In [1]:

"""
Script to read in various tables of valuation data and calculate the replacement 
cost of a structure inventory for later use in damage calculations

Required attributes in the structure inventory for valuation:
structure_id
point_id - CLARA point ID for matching up with census tract data
gbs_code - General Building Stock code (e.g., RES1, COM1)
sqft - total square footage
num_stories - used for RES1 assets only, assumed values of 1 or 2 stories (2 representing 2+)
garage - presence of an attached garage, used for RES1 assets only

The structure inventory CSV used for testing was a version of the CLARA 2023CMP
inventory that already had imputed values filled in for missing attributes;
as such, this is intended for use once any imputation has already been done.
Future modifications may be required at the point of reading in the structure 
inventory if the data set contains different column names tha should be renamed.

@author: David R Johnson, Purdue University
@date: May 2023
"""

import pandas as pd

def zero_pad(x, pad_length):
    return str(x).zfill(pad_length)

def get_structure_inventory(file_path):
    return pd.read_csv(file_path)

def get_res1_costs(file_path):
    return pd.read_csv(file_path)

def get_nonres1_value_data(file_path):
    return pd.read_csv(file_path)

def get_mhi_by_tract(file_path):
    mhi_by_tract = pd.read_csv(file_path)
    mhi_by_tract['tract_id'] = mhi_by_tract['tract_id'].astype(str)
    return mhi_by_tract.drop('year_usd', axis=1)

def get_garage_costs(file_path):
    return pd.read_csv(file_path)

def get_garage_weights(file_path):
    return pd.read_csv(file_path)

def get_means_class_weights(file_path):
    return pd.read_csv(file_path)

def get_res1_csvr(file_path):
    return pd.read_csv(file_path)

def get_inflation(file_path):
    return pd.read_csv(file_path)

def get_tracts(file_path):
    tracts = pd.read_csv(file_path)
    tracts['tract_id'] = tracts['state_fips_2010'].astype(str).str.zfill(2) + tracts['county_fips_2010'].astype(str).str.zfill(3) + tracts['tract_2010'].astype(str).str.zfill(6)
    return tracts[['point_id', 'tract_id']]

### replace base directory path, ensuing code does not assume a working directory
base_dir = "data\\valuation_data"
input_dir = base_dir + "\\valuation_data\\"

input_file = "garage_unit_cost.csv"
garage_costs = get_garage_costs(input_dir + input_file)

input_file = "garage_weights.csv"
garage_weights = get_garage_weights(input_dir + input_file)

input_file = "inflation_factors.csv"
inflation_factors = get_inflation(input_dir + input_file)

input_file = "nonres1_value_data.csv"
nonres1_value_data = get_nonres1_value_data(input_dir + input_file)

input_file = "res1_means_class_weights.csv"
means_class_weights = get_means_class_weights(input_dir + input_file)

input_file = "res1_csvr.csv"
res1_csvr = get_res1_csvr(input_dir + input_file)

input_file = "res1_replacement_costs.csv"
res1_costs = get_res1_costs(input_dir + input_file)

input_file = "mhi_by_tract.csv"
mhi_by_tract = get_mhi_by_tract(input_dir + input_file)

input_file = "tracts_by_point.csv"
census_tracts = get_tracts(input_dir + input_file)

# Merge avg_rep_cost_by_mhi_ratio
avg_rep_cost_by_mhi_ratio = pd.merge(means_class_weights, res1_costs, on="means_class", how="outer")
avg_rep_cost_by_mhi_ratio = avg_rep_cost_by_mhi_ratio.merge(inflation_factors, on="year_usd")
avg_rep_cost_by_mhi_ratio["weighted_value_per_sqft"] = avg_rep_cost_by_mhi_ratio["proportion"] * avg_rep_cost_by_mhi_ratio["value_per_sqft"] * avg_rep_cost_by_mhi_ratio["inflation_to_2020"]
avg_rep_cost_by_mhi_ratio = avg_rep_cost_by_mhi_ratio.groupby(["mhi_ratio", "stories"]).agg({"weighted_value_per_sqft": "sum"}).rename(columns={"weighted_value_per_sqft": "res1_value_per_sqft"}).reset_index()

# Merge avg_garage_cost_by_mhi_ratio
avg_garage_cost_by_mhi_ratio = pd.merge(garage_costs, garage_weights, on="num_cars")
avg_garage_cost_by_mhi_ratio = avg_garage_cost_by_mhi_ratio.merge(inflation_factors, on="year_usd")
avg_garage_cost_by_mhi_ratio["weighted_value"] = avg_garage_cost_by_mhi_ratio["value"] * avg_garage_cost_by_mhi_ratio["proportion"] * avg_garage_cost_by_mhi_ratio["inflation_to_2020"]
avg_garage_cost_by_mhi_ratio = avg_garage_cost_by_mhi_ratio.drop("proportion", axis=1)
avg_garage_cost_by_mhi_ratio = avg_garage_cost_by_mhi_ratio.merge(means_class_weights, on="means_class", how="outer")
avg_garage_cost_by_mhi_ratio["weighted_value"] = avg_garage_cost_by_mhi_ratio["weighted_value"] * avg_garage_cost_by_mhi_ratio["proportion"]
avg_garage_cost_by_mhi_ratio = avg_garage_cost_by_mhi_ratio.groupby(["mhi_ratio"]).agg({"weighted_value": "sum"}).rename(columns={"weighted_value": "garage_value"}).reset_index()



In [4]:
# Load structure inventory data
input_file = "SI_value_test.csv"
structures = pd.read_csv(input_dir + input_file)
#structures = structures[0:5]  # for testing only
# the following two lines may need adjustment if the input structure inventory has different column names
structures = structures[['structure_id', 'tract_id', 'gbs_code', 'imputed_total_sqft', 'imputed_num_stories', 'has_garage']]
structures = structures.rename(columns={"structure_id": "structure_id", "tract_id": "tract_id", "gbs_code": "str_gbs_code", "imputed_total_sqft": "sqft", "imputed_num_stories": "num_stories", "has_garage": "garage"})
structures["garage"] = structures["garage"].fillna(0)
structures.loc[structures["num_stories"] < 1, ["num_stories"]] = 1

# Construct the MHI ratio used for determining the proportion of houses of each RS Means construction class
#structures = structures.merge(census_tracts, on="point_id", how="left")
structures['tract_id'] = structures['tract_id'].astype(str)
structures = structures.merge(mhi_by_tract, on="tract_id", how="left")
structures["mhi"] = structures["mhi"].fillna(35000)
structures["mhi_ratio"] = structures["mhi"] / 35000

# Assign a threshold for the MHI ratio that determines the assumed proportion of housing stock with each RS Means class
for i in range(len(means_class_weights["mhi_ratio"].unique()), 0, -1):
    threshold = sorted(means_class_weights["mhi_ratio"].unique())[i-1]
    structures.loc[structures["mhi_ratio"] < threshold, "mhi_threshold"] = threshold

# Calculate a structure value as if all structures were single-family homes
# structures = structures.set_index("structure_id")
structures = structures.merge(avg_rep_cost_by_mhi_ratio, left_on=["mhi_threshold", "num_stories"], right_on=["mhi_ratio", "stories"], how="left")
structures = structures.merge(avg_garage_cost_by_mhi_ratio, left_on="mhi_threshold", right_on="mhi_ratio", how="left")
structures["res1_structure_value"] = structures["res1_value_per_sqft"] * structures["sqft"] + structures["garage"] * structures["garage_value"]

# Calculate a structure values as if all structures were not single-family homes
structures = structures.merge(nonres1_value_data, left_on="str_gbs_code", right_on="gbs_code", how="left")
structures = structures.merge(inflation_factors, left_on="value_year_usd", right_on="year_usd", how="left")
structures["nonres1_structure_value"] = structures["value_per_sqft"] * structures["sqft"] * structures["inflation_to_2020"]

# Assign the correct structure value depending on whether the asset is RES1 or not
structures["structure_value"] = structures.apply(lambda x: x["res1_structure_value"] if x["gbs_code"] == "RES1" else x["nonres1_structure_value"], axis=1)

# Replace CSVR with the appropriate residential CSVR for single-family homes (varies by 1- or 2-story homes)
structures = structures.merge(res1_csvr, on="num_stories", how="left")
structures["csvr"] = structures.apply(lambda x: x["res_csvr"] if x["str_gbs_code"] == "RES1" else x["csvr"], axis=1)
# Calculate contents value
structures["contents_value"] = structures["structure_value"] * structures["csvr"]

# Calculate inventory value
structures = structures.drop(["inflation_to_2020", "year_usd"], axis=1)
structures = structures.merge(inflation_factors, left_on="sales_year_usd", right_on="year_usd", how="left")
structures["inventory_value"] = structures["sales_per_sqft"] * structures["sqft"] * structures["inv_as_pct_of_sales"] / 100 * structures["inflation_to_2020"]

structures["replacement_cost"] = structures["structure_value"] + structures["contents_value"] + structures["inventory_value"]

structures = structures[["structure_id", "structure_value", "contents_value", "inventory_value", "replacement_cost"]]
structures.to_csv(input_dir + 'structure_values.csv', index=False)
